In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import validation_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from spreg import ML_Lag
from libpysal import weights
pd.set_option('display.float_format', lambda x: '%.10f' % x)

c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\libpysal\cg\alpha_shapes.py:39: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def nb_dist(x, y):
c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\libpysal\cg\alpha_shapes.py:165: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_faces(triangle):
c:\Users\84398\AppD

删除异常值 + 添加distanceTounderground。仅用作说明underground变量时使用。

In [3]:
df = pd.read_csv(r"D:\POI data\810_acc\reclass3_add_distanceTounderground.csv")
# 计算Price_per_square_meter的Q1和Q3
Q1 = df['Price_per_square_meter'].quantile(0.25)
Q3 = df['Price_per_square_meter'].quantile(0.75)

# 计算IQR
IQR = Q3 - Q1

# 过滤掉Price_per_square_meter的异常值
df = df[~((df['Price_per_square_meter'] < (Q1 - 1.5 * IQR)) | 
          (df['Price_per_square_meter'] > (Q3 + 1.5 * IQR)))]
          

#df_encoded = df
df_encoded = pd.get_dummies(df, columns=['property_type','tenure'], drop_first=True)

coords = df_encoded[['lon', 'lat']].values

w = weights.KNN.from_array(coords, k=5)
w.transform = 'r'  # row-standardization

def replace_variable_names(summary, column_names):
    for i, col in enumerate(column_names):
        summary = summary.replace(f"var_{i}", col)
    return summary

# Add constant to the predictors
X = df_encoded.drop(['price', 'address', 
        'Price_per_square_meter', 'id', 'lat', 'lon','borough','distance_to_center'], axis=1)
X = (X - X.mean()) / X.std()  # It's a good practice to standardize your variables

# Save the columns
X_cols = ["Constant"] + list(X.columns) # X.columns=43个


# Create a numpy array from the dataframe
X = pd.DataFrame(np.hstack([np.ones((X.shape[0], 1)), X])).values

y = df_encoded['Price_per_square_meter']
y = (y - y.mean()) / y.std()  # Standardize the target variable as well

# Fit a spatial lag model
ml_lag = ML_Lag(y.values[:, None], X, w)

# After the model is fitted, replace the variable names in the summary
summary = ml_lag.summary
print(ml_lag.summary)


c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
  warnings.warn(message)
c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_minimize.py:913: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:        3460
Mean dependent var  :      0.0000                Number of Variables   :          52
S.D. dependent var  :      1.0000                Degrees of Freedom    :        3408
Pseudo R-squared    :      0.6155
Spatial Pseudo R-squared:  0.4528
Sigma-square ML     :       0.387                Log likelihood        :   -3400.219
S.E of regression   :       0.622                Akaike info criterion :    6904.438
                                                 Schwarz criterion     :    7224.188

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------

In [4]:
# 定义数据
data = """
 CONSTANT      -0.0015885       0.0105780      -0.1501703       0.8806303
               var_1      -0.0620816       0.0205922      -3.0148184       0.0025713
               var_2       0.1467571       0.0165782       8.8523862       0.0000000
               var_3      -0.1088137       0.0172160      -6.3205162       0.0000000
               var_4       0.0466076       0.0140959       3.3064559       0.0009448
               var_5       0.1017905       0.0202067       5.0374618       0.0000005
               var_6       0.0076951       0.0109664       0.7017007       0.4828658
               var_7       0.1163962       0.0798170       1.4582883       0.1447611
               var_8       0.0163892       0.0359940       0.4553318       0.6488705
               var_9      -0.2113241       0.0761349      -2.7756523       0.0055091
              var_10       0.0739637       0.0485100       1.5247109       0.1273312
              var_11       0.0547489       0.0277602       1.9722106       0.0485856
              var_12       0.0373856       0.0414558       0.9018176       0.3671538
              var_13      -0.0089245       0.0121483      -0.7346232       0.4625690
              var_14      -0.0834001       0.0355046      -2.3489935       0.0188242
              var_15       0.0606230       0.0289766       2.0921406       0.0364259
              var_16      -0.0700855       0.0400395      -1.7504095       0.0800477
              var_17      -0.1263474       0.0286570      -4.4089531       0.0000104
              var_18       0.0107381       0.0168276       0.6381226       0.5233939
              var_19      -0.0583814       0.0738981      -0.7900254       0.4295129
              var_20      -0.0107699       0.0152585      -0.7058309       0.4802933
              var_21       0.1766434       0.0268932       6.5683215       0.0000000
              var_22       0.1245495       0.0430151       2.8954822       0.0037858
              var_23      -0.0687443       0.0331472      -2.0739089       0.0380878
              var_24      -0.0706102       0.0124227      -5.6839720       0.0000000
"""

# 初始化空列表
variable_list =  X_cols[0:25]
coefficient_list = []
std_error_list = []
z_statistic_list = []
probability_list = []

# 解析数据并填充列表
for line in data.strip().split("\n"):
    parts = line.split()
    coefficient_list.append(float(parts[1]))
    std_error_list.append(float(parts[2]))
    z_statistic_list.append(float(parts[3]))
    probability_list.append(float(parts[4]))

data = {
    'Variable': variable_list,
    'Coefficient': coefficient_list,
    'Std.Error': std_error_list,
    'z-Statistic': z_statistic_list,
    'Probability': probability_list
}

summary_df = pd.DataFrame(data)
#summary_df

In [5]:
df_sig = summary_df[summary_df['Probability'] < 0.05]
df_sig

,Variable,Coefficient,Std.Error,z-Statistic,Probability
1,bedrooms,-0.0620816000,0.0205922000,-3.0148184000,0.0025713000
2,bathrooms,0.1467571000,0.0165782000,8.8523862000,0.0000000000
3,size_square_meters,-0.1088137000,0.0172160000,-6.3205162000,0.0000000000
4,garden,0.0466076000,0.0140959000,3.3064559000,0.0009448000
5,AccommodationAccess15m,0.1017905000,0.0202067000,5.0374618000,0.0000005000
9,Commercial servicesAccess15m,-0.2113241000,0.0761349000,-2.7756523000,0.0055091000
11,HealthAccess15m,0.0547489000,0.0277602000,1.9722106000,0.0485856000
14,Bus stopAccess15m,-0.0834001000,0.0355046000,-2.3489935000,0.0188242000
15,CultureAccess15m,0.0606230000,0.0289766000,2.0921406000,0.0364259000
17,GamblingAccess15m,-0.1263474000,0.0286570000,-4.4089531000,0.0000104000


---

不加distance to center的reclass3

删除异常值

删除分类变量 的结果显著的变量和不删除分类变量差不多。
所以选择不删除分类变量的模型，然后只打印不包含分类变量的。

---

尝试删除多重共线性的变量之后

加distance to center+log不删除异常值

只打印不包含分类变量的。

用于最终



In [18]:
df = pd.read_csv(r"D:\POI data\810_acc\reclass3_add_distanceTounderground.csv")


# 选择解释变量
X = df.drop(['Price_per_square_meter', 'price',  'address', 'property_type',  'id', 'lat', 'lon','borough','tenure',], axis=1)

# 添加常数项
X['Constant'] = 1

# 计算VIF
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)


                                 Variable           VIF
0                                bedrooms  2.5878263039
1                               bathrooms  2.4779037354
2                      size_square_meters  2.6177195280
3                                  garden  1.1498618753
4                  AccommodationAccess15m  3.6156237252
5                       AirportsAccess15m  1.0422683592
6            Eating and drinkingAccess15m 55.2736145671
7                       ShoppingAccess15m  9.5066743383
8            Commercial servicesAccess15m 49.6172607495
9                      EducationAccess15m 21.8650079225
10                        HealthAccess15m  5.2918791068
11  Manufacturing and productionAccess15m 15.0652793723
12               Bodies of waterAccess15m  1.2910179279
13                      Bus stopAccess15m 11.1563471377
14                       CultureAccess15m  7.8610344107
15                 EntertainmentAccess15m 14.3995066899
16                      GamblingAccess15m  7.013

In [20]:
df = pd.read_csv(r"D:\POI data\810_acc\reclass3_add_distanceTounderground.csv")

# 确保所有的值都是正的
assert all(df['Price_per_square_meter'] > 0), "Some values are non-positive!"

# 对Price_per_square_meter进行对数变换
df['Price_per_square_meter'] = np.log(df['Price_per_square_meter'])


df_encoded = pd.get_dummies(df, columns=['property_type','tenure'], drop_first=True)
coords = df_encoded[['lon', 'lat']].values

w = weights.KNN.from_array(coords, k=5)
w.transform = 'r'  # row-standardization

def replace_variable_names(summary, column_names):
    for i, col in enumerate(column_names):
        summary = summary.replace(f"var_{i}", col)
    return summary

X = df_encoded.drop(['price', 'address', 
        'Price_per_square_meter', 'distance_to_center','id', 'lat', 'lon','borough','Eating and drinkingAccess15m',
       'Commercial servicesAccess15m','EducationAccess15m','Manufacturing and productionAccess15m',
        'Bus stopAccess15m','EntertainmentAccess15m','Public infrastructureAccess15m','TourismAccess15m'], axis=1)
X = (X - X.mean()) / X.std()  # It's a good practice to standardize your variables

# Save the columns
X_cols = ["Constant"] + list(X.columns) # X.columns=43个


# Create a numpy array from the dataframe
X = pd.DataFrame(np.hstack([np.ones((X.shape[0], 1)), X])).values

y = df_encoded['Price_per_square_meter']
y = (y - y.mean()) / y.std()  # Standardize the target variable as well

# Fit a spatial lag model
ml_lag = ML_Lag(y.values[:, None], X, w)

# After the model is fitted, replace the variable names in the summary
summary = ml_lag.summary
print(ml_lag.summary)



c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
  warnings.warn(message)
c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_minimize.py:913: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:        3737
Mean dependent var  :      0.0000                Number of Variables   :          45
S.D. dependent var  :      1.0000                Degrees of Freedom    :        3692
Pseudo R-squared    :      0.6519
Spatial Pseudo R-squared:  0.4867
Sigma-square ML     :       0.351                Log likelihood        :   -3498.505
S.E of regression   :       0.592                Akaike info criterion :    7087.009
                                                 Schwarz criterion     :    7367.181

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------

In [24]:
# 定义数据
data = """
 CONSTANT      -0.0005732       0.0096876      -0.0591644       0.9528212
               var_1      -0.0623231       0.0187375      -3.3261111       0.0008807
               var_2       0.2042926       0.0159546      12.8046405       0.0000000
               var_3      -0.1337969       0.0160376      -8.3426927       0.0000000
               var_4       0.0343178       0.0126493       2.7130147       0.0066674
               var_5       0.0903366       0.0175335       5.1522210       0.0000003
               var_6       0.0076845       0.0099088       0.7755271       0.4380282
               var_7       0.0928775       0.0244358       3.8008799       0.0001442
               var_8       0.0315316       0.0192060       1.6417540       0.1006410
               var_9      -0.0175691       0.0104721      -1.6777010       0.0934055
              var_10       0.0693496       0.0229311       3.0242579       0.0024924
              var_11      -0.1487618       0.0198566      -7.4918124       0.0000000
              var_12      -0.0055081       0.0133212      -0.4134839       0.6792521
              var_13      -0.0214802       0.0130038      -1.6518417       0.0985668
              var_14       0.0649159       0.0194981       3.3293516       0.0008705
              var_15      -0.0581076       0.0273226      -2.1267206       0.0334433
              var_16      -0.0857460       0.0112823      -7.6000287       0.0000000
"""

# 初始化空列表
variable_list =  X_cols[0:17]
coefficient_list = []
std_error_list = []
z_statistic_list = []
probability_list = []

# 解析数据并填充列表
for line in data.strip().split("\n"):
    parts = line.split()
    coefficient_list.append(float(parts[1]))
    std_error_list.append(float(parts[2]))
    z_statistic_list.append(float(parts[3]))
    probability_list.append(float(parts[4]))

data = {
    'Variable': variable_list,
    'Coefficient': coefficient_list,
    'Std.Error': std_error_list,
    'z-Statistic': z_statistic_list,
    'Probability': probability_list
}

summary_df = pd.DataFrame(data)

df_sig = summary_df[summary_df['Probability'] < 0.05]
df_sig


,Variable,Coefficient,Std.Error,z-Statistic,Probability
1,bedrooms,-0.0623231000,0.0187375000,-3.3261111000,0.0008807000
2,bathrooms,0.2042926000,0.0159546000,12.8046405000,0.0000000000
3,size_square_meters,-0.1337969000,0.0160376000,-8.3426927000,0.0000000000
4,garden,0.0343178000,0.0126493000,2.7130147000,0.0066674000
5,AccommodationAccess15m,0.0903366000,0.0175335000,5.1522210000,0.0000003000
7,ShoppingAccess15m,0.0928775000,0.0244358000,3.8008799000,0.0001442000
10,CultureAccess15m,0.0693496000,0.0229311000,3.0242579000,0.0024924000
11,GamblingAccess15m,-0.1487618000,0.0198566000,-7.4918124000,0.0000000000
14,SportsAccess15m,0.0649159000,0.0194981000,3.3293516000,0.0008705000
15,Underground entranceAccess15m,-0.0581076000,0.0273226000,-2.1267206000,0.0334433000


In [25]:
summary_df[summary_df['Probability'] >= 0.05]

,Variable,Coefficient,Std.Error,z-Statistic,Probability
0,Constant,-0.0005732000,0.0096876000,-0.0591644000,0.9528212000
6,AirportsAccess15m,0.0076845000,0.0099088000,0.7755271000,0.4380282000
8,HealthAccess15m,0.0315316000,0.0192060000,1.6417540000,0.1006410000
9,Bodies of waterAccess15m,-0.0175691000,0.0104721000,-1.6777010000,0.0934055000
12,Green parkAccess15m,-0.0055081000,0.0133212000,-0.4134839000,0.6792521000
13,Railway stationsAccess15m,-0.0214802000,0.0130038000,-1.6518417000,0.0985668000


---

---

---
加distance to center+log不删除异常值

只打印不包含分类变量的。

用于最终



In [11]:
import pandas as pd
import numpy as np
from pysal.lib import weights
from spreg import ML_Lag

df = pd.read_csv(r"D:\POI data\810_acc\reclass3_add_distanceTounderground.csv")

# 确保所有的值都是正的
assert all(df['Price_per_square_meter'] > 0), "Some values are non-positive!"

# 对Price_per_square_meter进行对数变换
df['Price_per_square_meter'] = np.log(df['Price_per_square_meter'])


df_encoded = pd.get_dummies(df, columns=['property_type','tenure'], drop_first=True)
coords = df_encoded[['lon', 'lat']].values

w = weights.KNN.from_array(coords, k=5)
w.transform = 'r'  # row-standardization

def replace_variable_names(summary, column_names):
    for i, col in enumerate(column_names):
        summary = summary.replace(f"var_{i}", col)
    return summary

# Add constant to the predictors
X = df_encoded.drop(['price', 'address', 
        'Price_per_square_meter', 'id', 'lat', 'lon','borough','distance_to_center'], axis=1)
X = (X - X.mean()) / X.std()  # It's a good practice to standardize your variables

# Save the columns
X_cols = ["Constant"] + list(X.columns) # X.columns=43个


# Create a numpy array from the dataframe
X = pd.DataFrame(np.hstack([np.ones((X.shape[0], 1)), X])).values

y = df_encoded['Price_per_square_meter']
y = (y - y.mean()) / y.std()  # Standardize the target variable as well

# Fit a spatial lag model
ml_lag = ML_Lag(y.values[:, None], X, w)

# After the model is fitted, replace the variable names in the summary
summary = ml_lag.summary
print(ml_lag.summary)


c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
  warnings.warn(message)
c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_minimize.py:913: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:        3737
Mean dependent var  :      0.0000                Number of Variables   :          53
S.D. dependent var  :      1.0000                Degrees of Freedom    :        3684
Pseudo R-squared    :      0.6551
Spatial Pseudo R-squared:  0.5172
Sigma-square ML     :       0.347                Log likelihood        :   -3463.060
S.E of regression   :       0.589                Akaike info criterion :    7032.119
                                                 Schwarz criterion     :    7362.099

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------

In [12]:
# 定义数据
data = """
   CONSTANT      -0.0005516       0.0096320      -0.0572663       0.9543331
               var_1      -0.0595600       0.0186669      -3.1906809       0.0014194
               var_2       0.1987164       0.0159223      12.4803820       0.0000000
               var_3      -0.1422025       0.0160338      -8.8689089       0.0000000
               var_4       0.0365132       0.0127066       2.8735712       0.0040586
               var_5       0.0660475       0.0183897       3.5915440       0.0003287
               var_6       0.0063593       0.0099269       0.6406099       0.5217762
               var_7       0.0560245       0.0714774       0.7838079       0.4331528
               var_8       0.1425297       0.0301521       4.7270296       0.0000023
               var_9      -0.0000432       0.0681672      -0.0006341       0.9994941
              var_10       0.0370543       0.0452269       0.8192974       0.4126168
              var_11       0.0025846       0.0222316       0.1162558       0.9074498
              var_12      -0.0728281       0.0376087      -1.9364671       0.0528105
              var_13      -0.0113742       0.0109760      -1.0362795       0.3000717
              var_14      -0.0528578       0.0323558      -1.6336405       0.1023344
              var_15       0.0566996       0.0271424       2.0889657       0.0367108
              var_16      -0.1731032       0.0368508      -4.6974023       0.0000026
              var_17      -0.1378329       0.0255877      -5.3866765       0.0000001
              var_18       0.0148606       0.0149972       0.9908961       0.3217363
              var_19      -0.0606662       0.0687754      -0.8820917       0.3777272
              var_20       0.0007436       0.0143070       0.0519727       0.9585505
              var_21       0.1339811       0.0245319       5.4614931       0.0000000
              var_22       0.2021680       0.0387943       5.2112874       0.0000002
              var_23      -0.0803056       0.0303237      -2.6482796       0.0080903
              var_24      -0.0811280       0.0114452      -7.0883605       0.0000000
"""

# 初始化空列表
variable_list =  X_cols[0:25]
coefficient_list = []
std_error_list = []
z_statistic_list = []
probability_list = []

# 解析数据并填充列表
for line in data.strip().split("\n"):
    parts = line.split()
    coefficient_list.append(float(parts[1]))
    std_error_list.append(float(parts[2]))
    z_statistic_list.append(float(parts[3]))
    probability_list.append(float(parts[4]))

data = {
    'Variable': variable_list,
    'Coefficient': coefficient_list,
    'Std.Error': std_error_list,
    'z-Statistic': z_statistic_list,
    'Probability': probability_list
}

summary_df = pd.DataFrame(data)

df_sig = summary_df[summary_df['Probability'] < 0.05]
df_sig

,Variable,Coefficient,Std.Error,z-Statistic,Probability
1,bedrooms,-0.0595600000,0.0186669000,-3.1906809000,0.0014194000
2,bathrooms,0.1987164000,0.0159223000,12.4803820000,0.0000000000
3,size_square_meters,-0.1422025000,0.0160338000,-8.8689089000,0.0000000000
4,garden,0.0365132000,0.0127066000,2.8735712000,0.0040586000
5,AccommodationAccess15m,0.0660475000,0.0183897000,3.5915440000,0.0003287000
8,ShoppingAccess15m,0.1425297000,0.0301521000,4.7270296000,0.0000023000
15,CultureAccess15m,0.0566996000,0.0271424000,2.0889657000,0.0367108000
16,EntertainmentAccess15m,-0.1731032000,0.0368508000,-4.6974023000,0.0000026000
17,GamblingAccess15m,-0.1378329000,0.0255877000,-5.3866765000,0.0000001000
21,SportsAccess15m,0.1339811000,0.0245319000,5.4614931000,0.0000000000


In [13]:
summary_df[summary_df['Probability'] >= 0.05]

,Variable,Coefficient,Std.Error,z-Statistic,Probability
0,Constant,-0.0005516000,0.0096320000,-0.0572663000,0.9543331000
6,AirportsAccess15m,0.0063593000,0.0099269000,0.6406099000,0.5217762000
7,Eating and drinkingAccess15m,0.0560245000,0.0714774000,0.7838079000,0.4331528000
9,Commercial servicesAccess15m,-0.0000432000,0.0681672000,-0.0006341000,0.9994941000
10,EducationAccess15m,0.0370543000,0.0452269000,0.8192974000,0.4126168000
11,HealthAccess15m,0.0025846000,0.0222316000,0.1162558000,0.9074498000
12,Manufacturing and productionAccess15m,-0.0728281000,0.0376087000,-1.9364671000,0.0528105000
13,Bodies of waterAccess15m,-0.0113742000,0.0109760000,-1.0362795000,0.3000717000
14,Bus stopAccess15m,-0.0528578000,0.0323558000,-1.6336405000,0.1023344000
18,Green parkAccess15m,0.0148606000,0.0149972000,0.9908961000,0.3217363000


---
加distance to center的reclass3
+log
删除异常值
只打印不包含分类变量的。



In [9]:
import pandas as pd
import numpy as np
from pysal.lib import weights
from spreg import ML_Lag

df = pd.read_csv(r"D:\POI data\810_acc\reclass3_add_distanceTounderground.csv")

# 确保所有的值都是正的
assert all(df['Price_per_square_meter'] > 0), "Some values are non-positive!"

# 对Price_per_square_meter进行对数变换
df['Price_per_square_meter'] = np.log(df['Price_per_square_meter'])

# 计算Price_per_square_meter的Q1和Q3
Q1 = df['Price_per_square_meter'].quantile(0.25)
Q3 = df['Price_per_square_meter'].quantile(0.75)

# 计算IQR
IQR = Q3 - Q1

# 过滤掉Price_per_square_meter的异常值
df = df[~((df['Price_per_square_meter'] < (Q1 - 1.5 * IQR)) | 
          (df['Price_per_square_meter'] > (Q3 + 1.5 * IQR)))]

df_encoded = pd.get_dummies(df, columns=['property_type','tenure'], drop_first=True)
coords = df_encoded[['lon', 'lat']].values

w = weights.KNN.from_array(coords, k=5)
w.transform = 'r'  # row-standardization

def replace_variable_names(summary, column_names):
    for i, col in enumerate(column_names):
        summary = summary.replace(f"var_{i}", col)
    return summary

# Add constant to the predictors
X = df_encoded.drop(['price', 'address', 
        'Price_per_square_meter', 'id', 'lat', 'lon','borough','distance_to_center'], axis=1)
X = (X - X.mean()) / X.std()  # It's a good practice to standardize your variables

# Save the columns
X_cols = ["Constant"] + list(X.columns) # X.columns=43个


# Create a numpy array from the dataframe
X = pd.DataFrame(np.hstack([np.ones((X.shape[0], 1)), X])).values

y = df_encoded['Price_per_square_meter']
y = (y - y.mean()) / y.std()  # Standardize the target variable as well

# Fit a spatial lag model
ml_lag = ML_Lag(y.values[:, None], X, w)

# After the model is fitted, replace the variable names in the summary
summary = ml_lag.summary
print(ml_lag.summary)


c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
  warnings.warn(message)
c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_minimize.py:913: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:        3667
Mean dependent var  :     -0.0000                Number of Variables   :          53
S.D. dependent var  :      1.0000                Degrees of Freedom    :        3614
Pseudo R-squared    :      0.6749
Spatial Pseudo R-squared:  0.5357
Sigma-square ML     :       0.327                Log likelihood        :   -3308.501
S.E of regression   :       0.572                Akaike info criterion :    6723.002
                                                 Schwarz criterion     :    7051.980

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------

In [10]:
# 定义数据
data = """
  CONSTANT      -0.0014911       0.0094464      -0.1578481       0.8745765
               var_1      -0.1386172       0.0185509      -7.4722779       0.0000000
               var_2       0.1128286       0.0156406       7.2138443       0.0000000
               var_3       0.0271797       0.0174036       1.5617266       0.1183524
               var_4       0.0241884       0.0125342       1.9297944       0.0536323
               var_5       0.0817908       0.0180076       4.5420221       0.0000056
               var_6       0.0074142       0.0097491       0.7605069       0.4469516
               var_7       0.1177798       0.0711862       1.6545321       0.0980194
               var_8       0.0897194       0.0303224       2.9588502       0.0030879
               var_9      -0.0379018       0.0673708      -0.5625847       0.5737177
              var_10       0.0295070       0.0441909       0.6677166       0.5043145
              var_11       0.0040803       0.0224837       0.1814780       0.8559924
              var_12      -0.0393491       0.0368249      -1.0685443       0.2852751
              var_13      -0.0080048       0.0107965      -0.7414288       0.4584335
              var_14      -0.0443226       0.0315038      -1.4068959       0.1594583
              var_15       0.0306150       0.0261878       1.1690561       0.2423810
              var_16      -0.1362084       0.0358127      -3.8033488       0.0001428
              var_17      -0.1261543       0.0254940      -4.9484011       0.0000007
              var_18       0.0064098       0.0147920       0.4333305       0.6647747
              var_19      -0.0716240       0.0674850      -1.0613323       0.2885389
              var_20      -0.0033575       0.0140170      -0.2395284       0.8106959
              var_21       0.1325175       0.0241055       5.4973952       0.0000000
              var_22       0.1678319       0.0374501       4.4814813       0.0000074
              var_23      -0.0929039       0.0293205      -3.1685674       0.0015319
              var_24      -0.0808877       0.0112222      -7.2078534       0.0000000
"""

# 初始化空列表
variable_list =  X_cols[0:25]
coefficient_list = []
std_error_list = []
z_statistic_list = []
probability_list = []

# 解析数据并填充列表
for line in data.strip().split("\n"):
    parts = line.split()
    coefficient_list.append(float(parts[1]))
    std_error_list.append(float(parts[2]))
    z_statistic_list.append(float(parts[3]))
    probability_list.append(float(parts[4]))

data = {
    'Variable': variable_list,
    'Coefficient': coefficient_list,
    'Std.Error': std_error_list,
    'z-Statistic': z_statistic_list,
    'Probability': probability_list
}

summary_df = pd.DataFrame(data)

df_sig = summary_df[summary_df['Probability'] < 0.05]
df_sig

,Variable,Coefficient,Std.Error,z-Statistic,Probability
1,bedrooms,-0.1386172000,0.0185509000,-7.4722779000,0.0000000000
2,bathrooms,0.1128286000,0.0156406000,7.2138443000,0.0000000000
5,AccommodationAccess15m,0.0817908000,0.0180076000,4.5420221000,0.0000056000
8,ShoppingAccess15m,0.0897194000,0.0303224000,2.9588502000,0.0030879000
16,EntertainmentAccess15m,-0.1362084000,0.0358127000,-3.8033488000,0.0001428000
17,GamblingAccess15m,-0.1261543000,0.0254940000,-4.9484011000,0.0000007000
21,SportsAccess15m,0.1325175000,0.0241055000,5.4973952000,0.0000000000
22,TourismAccess15m,0.1678319000,0.0374501000,4.4814813000,0.0000074000
23,Underground entranceAccess15m,-0.0929039000,0.0293205000,-3.1685674000,0.0015319000
24,distance_to_underground_entrance,-0.0808877000,0.0112222000,-7.2078534000,0.0000000000


---
不加distance to center的reclass3
+log
删除异常值
只打印不包含分类变量的。



In [4]:
import pandas as pd
import numpy as np
from pysal.lib import weights
from spreg import ML_Lag

df = pd.read_csv(r"D:\POI data\728_7acc\housing_hasId_reclass3_acc.csv")

# 确保所有的值都是正的
assert all(df['Price_per_square_meter'] > 0), "Some values are non-positive!"

# 对Price_per_square_meter进行对数变换
df['Price_per_square_meter'] = np.log(df['Price_per_square_meter'])

# 计算Price_per_square_meter的Q1和Q3
Q1 = df['Price_per_square_meter'].quantile(0.25)
Q3 = df['Price_per_square_meter'].quantile(0.75)

# 计算IQR
IQR = Q3 - Q1

# 过滤掉Price_per_square_meter的异常值
df = df[~((df['Price_per_square_meter'] < (Q1 - 1.5 * IQR)) | 
          (df['Price_per_square_meter'] > (Q3 + 1.5 * IQR)))]

df_encoded = pd.get_dummies(df, columns=['property_type','tenure'], drop_first=True)
coords = df_encoded[['lon', 'lat']].values

w = weights.KNN.from_array(coords, k=5)
w.transform = 'r'  # row-standardization

def replace_variable_names(summary, column_names):
    for i, col in enumerate(column_names):
        summary = summary.replace(f"var_{i}", col)
    return summary

# Add constant to the predictors
X = df_encoded.drop(['price', 'address', 
        'Price_per_square_meter', 'id', 'lat', 'lon','borough'], axis=1)
X = (X - X.mean()) / X.std()  # It's a good practice to standardize your variables

# Save the columns
X_cols = ["Constant"] + list(X.columns) # X.columns=43个


# Create a numpy array from the dataframe
X = pd.DataFrame(np.hstack([np.ones((X.shape[0], 1)), X])).values

y = df_encoded['Price_per_square_meter']
y = (y - y.mean()) / y.std()  # Standardize the target variable as well

# Fit a spatial lag model
ml_lag = ML_Lag(y.values[:, None], X, w)

# After the model is fitted, replace the variable names in the summary
summary = ml_lag.summary
print(ml_lag.summary)


c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
  warnings.warn(message)
c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_minimize.py:913: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:        3667
Mean dependent var  :     -0.0000                Number of Variables   :          52
S.D. dependent var  :      1.0000                Degrees of Freedom    :        3615
Pseudo R-squared    :      0.6736
Spatial Pseudo R-squared:  0.5148
Sigma-square ML     :       0.329                Log likelihood        :   -3334.882
S.E of regression   :       0.574                Akaike info criterion :    6773.765
                                                 Schwarz criterion     :    7096.535

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------

In [6]:
# 定义数据
data = """
 CONSTANT      -0.0015458       0.0094746      -0.1631471       0.8704026
               var_1      -0.1352276       0.0186057      -7.2680644       0.0000000
               var_2       0.1157587       0.0156790       7.3830400       0.0000000
               var_3       0.0290225       0.0174364       1.6644763       0.0960173
               var_4       0.0257252       0.0125706       2.0464500       0.0407121
               var_5       0.0756938       0.0180504       4.1934706       0.0000275
               var_6       0.0066689       0.0097773       0.6820836       0.4951861
               var_7       0.1490698       0.0712332       2.0927027       0.0363757
               var_8       0.0777029       0.0303799       2.5577064       0.0105365
               var_9      -0.0814923       0.0673429      -1.2101092       0.2262370
              var_10       0.0731547       0.0439261       1.6654036       0.0958323
              var_11       0.0099148       0.0225409       0.4398571       0.6600406
              var_12      -0.0277787       0.0369058      -0.7526925       0.4516347
              var_13      -0.0040764       0.0108189      -0.3767883       0.7063309
              var_14      -0.0678665       0.0314588      -2.1573162       0.0309810
              var_15       0.0331411       0.0262592       1.2620776       0.2069209
              var_16      -0.1541341       0.0358107      -4.3041369       0.0000168
              var_17      -0.1115937       0.0255143      -4.3737711       0.0000122
              var_18       0.0170177       0.0147591       1.1530317       0.2488974
              var_19      -0.0979138       0.0675522      -1.4494544       0.1472107
              var_20      -0.0099964       0.0140235      -0.7128292       0.4759515
              var_21       0.1314995       0.0241645       5.4418356       0.0000001
              var_22       0.1565460       0.0375299       4.1712323       0.0000303
              var_23      -0.0529368       0.0289198      -1.8304686       0.0671799
"""

# 初始化空列表
variable_list =  X_cols[0:24]
coefficient_list = []
std_error_list = []
z_statistic_list = []
probability_list = []

# 解析数据并填充列表
for line in data.strip().split("\n"):
    parts = line.split()
    coefficient_list.append(float(parts[1]))
    std_error_list.append(float(parts[2]))
    z_statistic_list.append(float(parts[3]))
    probability_list.append(float(parts[4]))

data = {
    'Variable': variable_list,
    'Coefficient': coefficient_list,
    'Std.Error': std_error_list,
    'z-Statistic': z_statistic_list,
    'Probability': probability_list
}

summary_df = pd.DataFrame(data)

df_sig = summary_df[summary_df['Probability'] < 0.05]
df_sig

,Variable,Coefficient,Std.Error,z-Statistic,Probability
1,bedrooms,-0.1352276000,0.0186057000,-7.2680644000,0.0000000000
2,bathrooms,0.1157587000,0.0156790000,7.3830400000,0.0000000000
4,garden,0.0257252000,0.0125706000,2.0464500000,0.0407121000
5,AccommodationAccess15m,0.0756938000,0.0180504000,4.1934706000,0.0000275000
7,Eating and drinkingAccess15m,0.1490698000,0.0712332000,2.0927027000,0.0363757000
8,ShoppingAccess15m,0.0777029000,0.0303799000,2.5577064000,0.0105365000
14,Bus stopAccess15m,-0.0678665000,0.0314588000,-2.1573162000,0.0309810000
16,EntertainmentAccess15m,-0.1541341000,0.0358107000,-4.3041369000,0.0000168000
17,GamblingAccess15m,-0.1115937000,0.0255143000,-4.3737711000,0.0000122000
21,SportsAccess15m,0.1314995000,0.0241645000,5.4418356000,0.0000001000


---
不加distance to center的reclass3

删除异常值
只打印不包含分类变量的。

用于最终discussion

In [ ]:
df = pd.read_csv(r"D:\POI data\728_7acc\housing_hasId_reclass3_acc.csv")
# 计算Price_per_square_meter的Q1和Q3
Q1 = df['Price_per_square_meter'].quantile(0.25)
Q3 = df['Price_per_square_meter'].quantile(0.75)

# 计算IQR
IQR = Q3 - Q1

# 过滤掉Price_per_square_meter的异常值
df = df[~((df['Price_per_square_meter'] < (Q1 - 1.5 * IQR)) | 
          (df['Price_per_square_meter'] > (Q3 + 1.5 * IQR)))]
        

df_encoded = pd.get_dummies(df, columns=['property_type','tenure'], drop_first=True)
coords = df_encoded[['lon', 'lat']].values

w = weights.KNN.from_array(coords, k=5)
w.transform = 'r'  # row-standardization

def replace_variable_names(summary, column_names):
    for i, col in enumerate(column_names):
        summary = summary.replace(f"var_{i}", col)
    return summary

# Add constant to the predictors
X = df_encoded.drop(['price', 'address', 
        'Price_per_square_meter', 'id', 'lat', 'lon','borough'], axis=1)
X = (X - X.mean()) / X.std()  # It's a good practice to standardize your variables

# Save the columns
X_cols = ["Constant"] + list(X.columns) # X.columns=43个


# Create a numpy array from the dataframe
X = pd.DataFrame(np.hstack([np.ones((X.shape[0], 1)), X])).values

y = df_encoded['Price_per_square_meter']
y = (y - y.mean()) / y.std()  # Standardize the target variable as well

# Fit a spatial lag model
ml_lag = ML_Lag(y.values[:, None], X, w)

# After the model is fitted, replace the variable names in the summary
summary = ml_lag.summary
print(ml_lag.summary)


c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
  warnings.warn(message)
c:\Users\84398\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_minimize.py:913: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:        3460
Mean dependent var  :      0.0000                Number of Variables   :          51
S.D. dependent var  :      1.0000                Degrees of Freedom    :        3409
Pseudo R-squared    :      0.6146
Spatial Pseudo R-squared:  0.4382
Sigma-square ML     :       0.389                Log likelihood        :   -3416.552
S.E of regression   :       0.623                Akaike info criterion :    6935.104
                                                 Schwarz criterion     :    7248.704

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------

In [7]:
# 定义数据
data = """
 CONSTANT      -0.0016328       0.0105979      -0.1540654       0.8775581
               var_1      -0.0571229       0.0206169      -2.7706917       0.0055937
               var_2       0.1495136       0.0165976       9.0081575       0.0000000
               var_3      -0.1097712       0.0172477      -6.3643840       0.0000000
               var_4       0.0485368       0.0141185       3.4378107       0.0005864
               var_5       0.0970760       0.0202329       4.7979330       0.0000016
               var_6       0.0072843       0.0109863       0.6630360       0.5073075
               var_7       0.1443129       0.0798229       1.8079138       0.0706199
               var_8       0.0032766       0.0360187       0.0909697       0.9275167
               var_9      -0.2557108       0.0758845      -3.3697384       0.0007524
              var_10       0.1135916       0.0480901       2.3620591       0.0181737
              var_11       0.0624509       0.0277902       2.2472244       0.0246257
              var_12       0.0504897       0.0414675       1.2175720       0.2233867
              var_13      -0.0051055       0.0121537      -0.4200770       0.6744292
              var_14      -0.1055140       0.0353869      -2.9817223       0.0028663
              var_15       0.0644189       0.0290212       2.2197186       0.0264379
              var_16      -0.0831200       0.0400268      -2.0766055       0.0378380
              var_17      -0.1135688       0.0286493      -3.9641082       0.0000737
              var_18       0.0198260       0.0167648       1.1825941       0.2369701
              var_19      -0.0806298       0.0738965      -1.0911178       0.2752210
              var_20      -0.0163587       0.0152522      -1.0725508       0.2834727
              var_21       0.1761179       0.0269340       6.5388752       0.0000000
              var_22       0.1120045       0.0430500       2.6017316       0.0092754
              var_23      -0.0332216       0.0326071      -1.0188449       0.3082766
"""

# 初始化空列表
variable_list =  X_cols[0:24]
coefficient_list = []
std_error_list = []
z_statistic_list = []
probability_list = []

# 解析数据并填充列表
for line in data.strip().split("\n"):
    parts = line.split()
    coefficient_list.append(float(parts[1]))
    std_error_list.append(float(parts[2]))
    z_statistic_list.append(float(parts[3]))
    probability_list.append(float(parts[4]))

data = {
    'Variable': variable_list,
    'Coefficient': coefficient_list,
    'Std.Error': std_error_list,
    'z-Statistic': z_statistic_list,
    'Probability': probability_list
}

summary_df = pd.DataFrame(data)
#summary_df

In [102]:

df_sig = summary_df[summary_df['Probability'] < 0.05]
df_sig


,Variable,Coefficient,Std.Error,z-Statistic,Probability
1,bedrooms,-0.0571229000,0.0206169000,-2.7706917000,0.0055937000
2,bathrooms,0.1495136000,0.0165976000,9.0081575000,0.0000000000
3,size_square_meters,-0.1097712000,0.0172477000,-6.3643840000,0.0000000000
4,garden,0.0485368000,0.0141185000,3.4378107000,0.0005864000
5,AccommodationAccess15m,0.0970760000,0.0202329000,4.7979330000,0.0000016000
9,Commercial servicesAccess15m,-0.2557108000,0.0758845000,-3.3697384000,0.0007524000
10,EducationAccess15m,0.1135916000,0.0480901000,2.3620591000,0.0181737000
11,HealthAccess15m,0.0624509000,0.0277902000,2.2472244000,0.0246257000
14,Bus stopAccess15m,-0.1055140000,0.0353869000,-2.9817223000,0.0028663000
15,CultureAccess15m,0.0644189000,0.0290212000,2.2197186000,0.0264379000


In [103]:
df_not_sig = summary_df[summary_df['Probability'] >= 0.05]
df_not_sig


,Variable,Coefficient,Std.Error,z-Statistic,Probability
0,Constant,-0.0016328000,0.0105979000,-0.1540654000,0.8775581000
6,AirportsAccess15m,0.0072843000,0.0109863000,0.6630360000,0.5073075000
7,Eating and drinkingAccess15m,0.1443129000,0.0798229000,1.8079138000,0.0706199000
8,ShoppingAccess15m,0.0032766000,0.0360187000,0.0909697000,0.9275167000
12,Manufacturing and productionAccess15m,0.0504897000,0.0414675000,1.2175720000,0.2233867000
13,Bodies of waterAccess15m,-0.0051055000,0.0121537000,-0.4200770000,0.6744292000
18,Green parkAccess15m,0.0198260000,0.0167648000,1.1825941000,0.2369701000
19,Public infrastructureAccess15m,-0.0806298000,0.0738965000,-1.0911178000,0.2752210000
20,Railway stationsAccess15m,-0.0163587000,0.0152522000,-1.0725508000,0.2834727000
23,Underground entranceAccess15m,-0.0332216000,0.0326071000,-1.0188449000,0.3082766000


In [79]:
from esda.moran import Moran

# Compute Moran's I
moran = Moran(y, w)

# Print results
print("Moran's I: ", moran.I)
print("Standard deviation: ", moran.p_sim)
print("p-value: ", moran.p_norm)

Moran's I:  0.6529264876814679
Standard deviation:  0.001
p-value:  0.0


----
之前没有删除异常值的结果

In [3]:



variable_list = ['Constant',
 'bedrooms',
 'bathrooms',
 'size_square_meters',
 'garden',
 'AccommodationAccess15m',
 'AirportsAccess15m',
 'Eating and drinkingAccess15m',
 'ShoppingAccess15m',
 'Commercial servicesAccess15m',
 'EducationAccess15m',
 'HealthAccess15m',
 'Manufacturing and productionAccess15m',
 'Bodies of waterAccess15m',
 'Bus stopAccess15m',
 'CultureAccess15m',
 'EntertainmentAccess15m',
 'GamblingAccess15m',
 'Green parkAccess15m',
 'Public infrastructureAccess15m',
 'Railway stationsAccess15m',
 'SportsAccess15m',
 'TourismAccess15m',
 'Underground entranceAccess15m']

coefficient_list = [-0.0020700, -0.0827614, 0.1454069, 0.0395196, 0.0255961, 0.0322849, -0.0018209, -0.0065549,
                    0.2006695, 0.0572788, 0.0439343, -0.0159167, -0.0794472, -0.0204476, -0.0584362, 0.0533953,
                    -0.2270312, -0.1068069, 0.0143506, -0.0195311, -0.0117919, 0.0804087, 0.1801286, -0.0099796]

std_error_list = [0.0093591, 0.0181344, 0.0154669, 0.0155968, 0.0123449, 0.0177890, 0.0096446, 0.0692324,
                  0.0296085, 0.0659906, 0.0435382, 0.0215914, 0.0365472, 0.0106649, 0.0312769, 0.0263613,
                  0.0358976, 0.0247798, 0.0144669, 0.0666791, 0.0138759, 0.0237574, 0.0378467, 0.0289957]

z_statistic_list = [-0.2211788, -4.5637872, 9.4011745, 2.5338291, 2.0734158, 1.8148800, -0.1888013, -0.0946799,
                    6.7774330, 0.8679846, 1.0090978, -0.7371792, -2.1738270, -1.9172731, -1.8683477, 2.0255154,
                    -6.3244056, -4.3102423, 0.9919592, -0.2929113, -0.8498120, 3.3845821, 4.7594311, -0.3441742]

probability_list = [0.8249532, 0.0000050, 0.0000000, 0.0112824, 0.0381336, 0.0695424, 0.8502486, 0.9245691,
                    0.0000000, 0.3854028, 0.3129277, 0.4610134, 0.0297181, 0.0552033, 0.0617136, 0.0428145,
                    0.0000000, 0.0000163, 0.3212174, 0.7695900, 0.3954296, 0.0007129, 0.0000019, 0.7307153]

data = {
    'Variable': variable_list,
    'Coefficient': coefficient_list,
    'Std.Error': std_error_list,
    'z-Statistic': z_statistic_list,
    'Probability': probability_list
}

summary_df = pd.DataFrame(data)
summary_df


,Variable,Coefficient,Std.Error,z-Statistic,Probability
0,Constant,-0.0020700000,0.0093591000,-0.2211788000,0.8249532000
1,bedrooms,-0.0827614000,0.0181344000,-4.5637872000,0.0000050000
2,bathrooms,0.1454069000,0.0154669000,9.4011745000,0.0000000000
3,size_square_meters,0.0395196000,0.0155968000,2.5338291000,0.0112824000
4,garden,0.0255961000,0.0123449000,2.0734158000,0.0381336000
5,AccommodationAccess15m,0.0322849000,0.0177890000,1.8148800000,0.0695424000
6,AirportsAccess15m,-0.0018209000,0.0096446000,-0.1888013000,0.8502486000
7,Eating and drinkingAccess15m,-0.0065549000,0.0692324000,-0.0946799000,0.9245691000
8,ShoppingAccess15m,0.2006695000,0.0296085000,6.7774330000,0.0000000000
9,Commercial servicesAccess15m,0.0572788000,0.0659906000,0.8679846000,0.3854028000


In [4]:

df_sig = summary_df[summary_df['Probability'] < 0.05]
df_sig


,Variable,Coefficient,Std.Error,z-Statistic,Probability
1,bedrooms,-0.0827614000,0.0181344000,-4.5637872000,0.0000050000
2,bathrooms,0.1454069000,0.0154669000,9.4011745000,0.0000000000
3,size_square_meters,0.0395196000,0.0155968000,2.5338291000,0.0112824000
4,garden,0.0255961000,0.0123449000,2.0734158000,0.0381336000
8,ShoppingAccess15m,0.2006695000,0.0296085000,6.7774330000,0.0000000000
12,Manufacturing and productionAccess15m,-0.0794472000,0.0365472000,-2.1738270000,0.0297181000
15,CultureAccess15m,0.0533953000,0.0263613000,2.0255154000,0.0428145000
16,EntertainmentAccess15m,-0.2270312000,0.0358976000,-6.3244056000,0.0000000000
17,GamblingAccess15m,-0.1068069000,0.0247798000,-4.3102423000,0.0000163000
21,SportsAccess15m,0.0804087000,0.0237574000,3.3845821000,0.0007129000


In [5]:

df_not_sig = summary_df[summary_df['Probability'] >= 0.05]
df_not_sig

,Variable,Coefficient,Std.Error,z-Statistic,Probability
0,Constant,-0.0020700000,0.0093591000,-0.2211788000,0.8249532000
5,AccommodationAccess15m,0.0322849000,0.0177890000,1.8148800000,0.0695424000
6,AirportsAccess15m,-0.0018209000,0.0096446000,-0.1888013000,0.8502486000
7,Eating and drinkingAccess15m,-0.0065549000,0.0692324000,-0.0946799000,0.9245691000
9,Commercial servicesAccess15m,0.0572788000,0.0659906000,0.8679846000,0.3854028000
10,EducationAccess15m,0.0439343000,0.0435382000,1.0090978000,0.3129277000
11,HealthAccess15m,-0.0159167000,0.0215914000,-0.7371792000,0.4610134000
13,Bodies of waterAccess15m,-0.0204476000,0.0106649000,-1.9172731000,0.0552033000
14,Bus stopAccess15m,-0.0584362000,0.0312769000,-1.8683477000,0.0617136000
18,Green parkAccess15m,0.0143506000,0.0144669000,0.9919592000,0.3212174000


---